In [1]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt

In [6]:
def MSM1(dnbr, 
          plot=False, 
          save=False):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = np.array([0.67])
    b_3 = 0.7
    
    

    dnbr_flat = dnbr.values.flatten()
    pixarr = np.empty((dnbr_flat.shape[0]), dtype= float)
    pixarr[:] = np.nan
    x = np.where(np.isnan(dnbr_flat) == False)
        
    # Getting the data from dnbr_flat
    dnbrdata = dnbr_flat[x]
        
    # Pulling tif metadata
    metadata = dnbr.attrs
    x_1 = float(metadata.get('X1'))
    x_3 = float(metadata.get('X3'))
    s = np.multiply(dnbrdata, b_2)
    s1 = np.multiply(s, val)
    # s1 multiplication assumes when function is called that val is defined globally in code.
    
    # Per pixel analysis, probability stored in empty list p1
    p1 = []
    for i in s1:
        lnx =  b + (b_1 * x_1 * val) + i + (b_3 * x_3 * val)
        prob = (e ** lnx) / (1.0 + e ** lnx)
        p1.append(prob)
       
    # Append probability array, flatten data, reshape into image
    prob_arr = np.array(p1)
    p_final = prob_arr.flatten()
    dist_list.append(p_final)
    pixarr[x] = p_final
    pixarrImg = pixarr.reshape((dnbr.shape[1], dnbr.shape[2]))
    out_dnbr = xr.Dataset()
    out_dnbr = xr.DataArray(pixarrImg, dims = ('y', 'x'),
                            coords = {'x': dnbr.coords['x'],
                                      'y': dnbr.coords['y']})
    
    basin = '17346'

    if plot == True:
        out_dnbr.plot(
            cmap = 'gnuplot',
            vmax = 1,
            vmin = 0)
        plt.title(f"Basin {basin} Per Pixel Initiation Probablilty, R = {int(val*4)}mm/hr")
    
    if save == True:
        outname = f"{basin}_ppp_{int(val*4)}mmhr.tif"
        out_dnbr.rio.to_raster(os.path.join(ppp_out, outname), driver='GTIFF')
        print(f"Basin {basin} {int(val*4)}mm/hr per pixel analysis saved as geotiff")
        
        print(f"Basin {basin} {int(val*4)}mm/hr per pixel analysis complete")
    return out_dnbr

In [9]:
R = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
dNBR = rxr.open_rasterio(r'D:/imagery/masters/output/basin_17346_dNBR.tif')

dist_list = []

for val in R:
    MSM1(dNBR, False, False)


In [8]:
print(dist_list)

[array([0.09549278, 0.09174125, 0.09814919, ..., 0.22353427, 0.21521952,
       0.10168549]), array([0.14334435, 0.13625577, 0.14839053, ..., 0.38930124, 0.37402355,
       0.15513941]), array([0.20961746, 0.19766879, 0.21812941, ..., 0.58532861, 0.56555975,
       0.22951367]), array([0.29594572, 0.27785745, 0.30876042, ..., 0.75761068, 0.73933383,
       0.32579383]), array([0.3998414 , 0.37535761, 0.41696596, ..., 0.87375349, 0.86071731,
       0.43942818]), array([0.51360351, 0.48413387, 0.53380805, ..., 0.93874505, 0.93086235,
       0.55978882]), array([0.62597339, 0.59443457, 0.64705493, ..., 0.97137513, 0.96703421,
       0.67350772]), array([0.72622123, 0.69596207, 0.74588767, ..., 0.98686657, 0.98459465,
       0.76992386])]
